In [1]:
import json
import requests
import pymysql.cursors
from datetime import datetime
import sys
import time
import numpy as np
import pandas as pd
from fake_useragent import UserAgent
import time 
ua = UserAgent()

In [2]:
def conectaBanco():
	con = pymysql.connect(host = '127.0.0.1',
                      	      user = 'root',
                              passwd='root',
                              db = 'Prefeitura',
							  use_unicode=True,
							  charset="utf8",
                              cursorclass=pymysql.cursors.DictCursor)
	return con

In [3]:
def limpaDadosBanco(nomeTabela):
	con = conectaBanco()
	cursor = con.cursor()

	sqlquery = "SET FOREIGN_KEY_CHECKS = 0;"
	cursor.execute(sqlquery)
	con.commit()

	sqlquery = "TRUNCATE "+nomeTabela+";"
	cursor.execute(sqlquery)
	con.commit()
	print("DADOS DA TABELA "+nomeTabela+" EXCLUIDOS")
	con.close()

In [4]:
def salvaCidadeJson():
    #recupera cidades da api do IBGE
    link = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios"
    header = {'User-Agent':str(ua.chrome)}
    cidades = []

    marcador = True
    while(marcador):
        marcador = False
        try:
            r = requests.get(link,headers = header)
            if r.status_code == 200:

                reddit_data = json.loads(r.content)

                #Salva as cidades recuperadas em um arquivo cidades.json
                with open('cidades/cidades.json', 'w') as json_file:  
                    json.dump(reddit_data, json_file)

            else:
                print("")
                print("= HTTP Error 503. The service is unavailable. =")
        except requests.ConnectionError:
            print('Sem conexao com internet, aguardando conexao')
            marcador = True

In [5]:
def lerCidadesJson():
    # lemos o JSON em disco
    conteudo = open('cidades/cidades.json').read()
    cidades = []
    listajson = json.loads(conteudo)

    for cidade in listajson:
        cidades.append([cidade['id'],cidade['nome']])
    return cidades

In [6]:
def insereCidadesBanco(cidades):
    con = conectaBanco()
    for cidade in cidades:
        try:
            cidade[1] = cidade[1].replace("'","\\'")
            sql = "INSERT INTO cidade (idCidade,nome) values ("+str(cidade[0])+",'"+str(cidade[1])+"')"
    #         print(sql)
            cursor = con.cursor()
            cursor.execute(sql)
            con.commit()
        except pymysql.IntegrityError as e:
            print(str(e))
    con.close()

In [7]:
def salvaPesquisaJson():
    link = "https://servicodados.ibge.gov.br/api/v1/pesquisas/"
    header = {'User-Agent':str(ua.chrome)}

    marcador = True
    while(marcador):
        marcador = False
        try:
            r = requests.get(link,headers=header)
            if r.status_code == 200:

                reddit_data = json.loads(r.content)

                with open('pesquisas/pesquisas.json', 'w') as json_file:  
                    json.dump(reddit_data, json_file)

            else:
                print(idPesquisa)
                print("= HTTP Error 503. The service is unavailable. =")
        except requests.ConnectionError:
            print('Sem conexao com internet, aguardando conexao')
            marcador = True

In [8]:
def lerPesquisasJson():
    # lemos o JSON em disco
    conteudo = open('pesquisas/pesquisas.json').read()
    pesquisas = []
    listajson = json.loads(conteudo)

    for pesquisa in listajson:
        pesquisas.append([pesquisa['id'],pesquisa['nome']])
    return pesquisas

In [9]:
def inserePesquisasBanco(pesquisas):
    con = conectaBanco()
    for pesquisa in pesquisas:
        try:
            pesquisa[1] = pesquisa[1].replace("'","\\'")
            sql = "INSERT INTO pesquisa (idpesquisa,nome) values ("+str(pesquisa[0])+",'"+str(pesquisa[1])+"')"
    #         print(sql)
            cursor = con.cursor()
            cursor.execute(sql)
            con.commit()
        except pymysql.IntegrityError as e:
            print(str(e))
    con.close()

In [10]:
def salvaIndicadoresJson(pesquisas):
    for pesquisa in pesquisas:
        link = "https://servicodados.ibge.gov.br/api/v1/pesquisas/"+str(pesquisa[0])+"/indicadores"
        header = {'User-Agent':str(ua.chrome)}

        marcador = True
        while(marcador):
            marcador = False
            try:
                r = requests.get(link,headers=header)
                if r.status_code == 200:

                    reddit_data = json.loads(r.content)


                    with open('pesquisas/indicadores/'+str(pesquisa[0])+'.json', 'w') as json_file:  
                        json.dump(reddit_data, json_file)

            except requests.ConnectionError:
                print('Sem conexao com internet, aguardando conexao')
                marcador = True

In [11]:
def lerIndicadoresFilhos(listajson,indicadores):
    for indicador in listajson:
        indicadores.append([indicador['id'],indicador['indicador']])
        if(len(indicador['children'])>0):
            lerIndicadoresFilhos(indicador['children'],indicadores)

In [12]:
def lerIndicadoresJson(pesquisas):
    indicadores = []
    for pesquisa in pesquisas:
        try:
            conteudo = open('pesquisas/indicadores/'+str(pesquisa[0])+'.json').read()
            listajson = json.loads(conteudo)

            lerIndicadoresFilhos(listajson,indicadores)
        except FileNotFoundError:
            print('A pesquisa '+str(pesquisa[0])+' nao possui indicadores')
    return indicadores

In [13]:
def insereIndicadoresBanco(indicadores):
    con = conectaBanco()
    for indicador in indicadores:
        try:
            indicador[1] = indicador[1].replace("'","\\'")
            sql = "INSERT INTO indicador (idindicador,indicador) values ("+str(indicador[0])+",'"+str(indicador[1])+"')"
    #         print(sql)
            cursor = con.cursor()
            cursor.execute(sql)
            con.commit()
        except pymysql.IntegrityError as e:
            print(str(e))
    con.close()

In [14]:
def retornaIndicadoresASeremPesquisados():
    return open('pesquisas/indicadores/indicadores.txt').read()

In [15]:
def salvaHistoricoJson(codIBGE,indicadores):
#     codIBGE = 314710
    link = "https://servicodados.ibge.gov.br/api/v1/pesquisas/indicadores/"+indicadores+"/resultados/"+str(codIBGE)

    header = {'User-Agent':str(ua.chrome)}
    contador = 0
    metricas = []

    r = requests.get(link,headers = header)
    if r.status_code == 200:
        reddit_data = json.loads(r.content)

         #Salva as cidades recuperadas em um arquivo cidades.json
        with open('historico/'+str(codIBGE)+'.json', 'w') as json_file:  
            json.dump(reddit_data, json_file)

    else:
        print(r.status_code)
        print("= HTTP Error 503. The service is unavailable. =")

In [16]:
def salvaHistoricoParaCidadesJson(cidades,indicadores):
    for cidade in cidades:
        marcador = True
        while(marcador):
            marcador = False
            try:
                salvaHistoricoJson(cidade[0],indicadores)
            except:
                print(cidade)
                marcador = True

In [17]:
def lerHistoricoJson(cidades):
    historico = []

    for cidade in cidades:
        conteudo = open('historico/'+str(cidade[0])+'.json').read()
        listajson = json.loads(conteudo)
        for pesq in listajson:    
            for item in pesq['res'][0]['res']:
                historico.append([cidade[0],pesq['id'],item,pesq['res'][0]['res'][item]])
    return historico

In [18]:
def insereHistoricoBanco(historico):
    con = conectaBanco()
    for pesq in historico:
        try:
            pesq[3] = pesq[3].replace("'","\\'")
            sql = "INSERT INTO historico (idcidade,idindicador,ano,valor) values ("+str(pesq[0])+","+str(pesq[1])+","+str(pesq[2])+",'"+str(pesq[3])+"')"
    #         print(sql)
            cursor = con.cursor()
            cursor.execute(sql)
            con.commit()
        except pymysql.IntegrityError as e:
            print(str(e))
    con.close()

## Main

## Cidades
* Recuperando cidades salvando num json
* Lendo json salvando cidades num vetor
* Adicionando cidades no banco

In [19]:
inicioTime = time.time()

In [20]:
limpaDadosBanco('cidade')
limpaDadosBanco('pesquisa')
limpaDadosBanco('indicador')
limpaDadosBanco('historico')

DADOS DA TABELA cidade EXCLUIDOS
DADOS DA TABELA pesquisa EXCLUIDOS
DADOS DA TABELA indicador EXCLUIDOS
DADOS DA TABELA historico EXCLUIDOS


In [21]:
salvaCidadeJson()
cidades = lerCidadesJson()
insereCidadesBanco(cidades)

#Contagem de tempo
cidadeTime = time.time()

## Pesquisas
* Recuperando pesquisas salvando num json
* Lendo json salvando pesquisas num vetor
* Adicionando pesquisas no banco

In [22]:
salvaPesquisaJson()
pesquisas = lerPesquisasJson()
inserePesquisasBanco(pesquisas)

#Contagem de tempo
pesquisaTime = time.time()

## Indicadores
* Recuperando indicadoress salvando num json
* Lendo json salvando indicadores num vetor
* Adicionando indicadores no banco

In [23]:
salvaIndicadoresJson(pesquisas)
indicadores = lerIndicadoresJson(pesquisas)
insereIndicadoresBanco(indicadores)

#Contagem de tempo
indicadoresTime = time.time()

A pesquisa 1 nao possui indicadores
A pesquisa 24 nao possui indicadores
A pesquisa 45 nao possui indicadores
A pesquisa 10072 nao possui indicadores
A pesquisa 10079 nao possui indicadores
A pesquisa 10085 nao possui indicadores


## Histórico
* Recuperando indicadoress a serem pesquisados do banco (Os indicadores a serem pesquisados estão em src/pesquisa/indicadores/indicarores.txt)
* Recuperando historico e salvando num json em src/historico (o nome de cada json sera o id de cada cidade)
* Lendo json dos historicos e salvando em um vetor
* Adicionando o historico no banco

In [19]:
cidades = lerCidadesJson()

In [26]:
dfCidades = pd.DataFrame(cidades)

In [29]:
dfCidades[dfCidades[0] == 2909307]

,0,1
1938,2909307,Correntina


In [32]:
cidades = cidades[1938:]

In [34]:
historico = lerHistoricoJson(cidades)

In [35]:
# indicadoresASeremPesquisados = retornaIndicadoresASeremPesquisados()
# salvaHistoricoParaCidadesJson(cidades,indicadoresASeremPesquisados)
# historico = lerHistoricoJson(cidades)
insereHistoricoBanco(historico)

#Contagem de tempo
# historicoTime = time.time()

In [ ]:
#Contagem de tempo
fimTime = time.time()

In [ ]:
print("Tempo gasto total"+str(fimTime - inicioTime))